In [1]:
%load_ext autoreload
%autoreload 2

import mock_ledger 
import S4Mock_io
import glob
import geometry
import numpy as np
from astropy.table import Table,vstack
from   desitarget.sv3.sv3_targetmask import desi_mask, bgs_mask, mws_mask

In [2]:
nights    = [x.split('/')[-1] for x in sorted(glob.glob('/global/cscratch1/sd/mjwilson/altmtls/ledger/initial/Univ000/fa/SV3' + '/*'))]
nights

['20210406',
 '20210407',
 '20210408',
 '20210409',
 '20210410',
 '20210411',
 '20210412',
 '20210413',
 '20210414',
 '20210415',
 '20210416',
 '20210417',
 '20210418',
 '20210420',
 '20210422',
 '20210429',
 '20210501',
 '20210502',
 '20210503',
 '20210504',
 '20210505',
 '20210506',
 '20210507',
 '20210508',
 '20210509',
 '20210510',
 '20210511',
 '20210512',
 '20210514',
 '20210529']

In [3]:
from desimodel.footprint import is_point_in_desi, tiles2pix
from desitarget.geomask import pixarea2nside

#read targets in 
#get tiles
tiles = S4Mock_io.read_sv3tiles()
# closest nside to DESI tile area of ~7 deg
nside = pixarea2nside(7.)
# ADM determine the pixels that touch the tiles.
pixlist = tiles2pix(nside, tiles=tiles)
#read in mxxl
mxxl =S4Mock_io.read_mxxl(small=False,nside=nside)
#read in our mxxl targets but having this nside and this pixlist 
targets = mxxl[np.in1d(mxxl['HPX'],pixlist)]
#restrict only to targets in the requested tiles...
ii = is_point_in_desi(tiles, targets["RA"], targets["DEC"])
targets = targets[ii]
#now get pixlist in nside=32
pix32 = geometry.radec2pix(targets,nside=32)
targets['HPX']=pix32

In [4]:
#sv3 pix now more accurate as not just using central ra and dec 
pix4ledger = geometry.radec2pix(targets,nside=32,unique=True).data
pix4ledger

array([2286, 2287, 2333, 2334, 2335, 2356, 2357, 2376, 2378, 2400, 2447,
       2453, 2454, 2455, 2457, 2458, 2459, 2460, 2461, 2462, 2469, 2471,
       2480, 2481, 2482, 2483, 2484, 2495, 2496, 2498, 2504, 2536, 2537,
       2538, 2539, 2549, 2551, 2557, 2559, 2592, 2594, 2595, 2600, 2601,
       2602, 2628, 2629, 2630, 2631, 2679, 2685, 2803, 2806, 2807, 2809,
       2812, 2813, 2837, 2850, 2851, 2856, 2857, 2880, 2881, 2901, 2903,
       2978, 2984, 3834, 3835, 3838, 3839, 4010, 4011, 6398, 6399, 6477,
       6478, 6479, 6481, 6482, 6483, 6484, 6486, 6488, 6489, 6490, 6491,
       6492, 6493, 6494, 6495, 6500, 6501, 6503, 6509, 6511, 6512, 6513,
       6516, 6517, 6520, 6521, 6522, 6523, 6562, 6563, 6568, 6569, 6570,
       6571, 6572, 6574, 6608, 6609, 6740, 6741, 6743, 6809, 6811, 6812,
       6814, 6815, 6833, 6836, 6837, 6912, 6914, 7006, 7007, 7028, 7029,
       7030, 7031, 7037])

In [15]:
real_ledger = vstack([Table.read(x) for x in glob.glob('/global/cscratch1/sd/mjwilson/S4MOCK/SV3REAL/SV3REALLEDGER/bright/sv3mtl-bright-hp-*.ecsv')])

In [40]:
for i in range(len(pix4ledger)):
    init = mock_ledger.create_mock_ledger_hp(outdir='/global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3', healpix=pix4ledger[i], mxxl=targets, overwrite=False)


Selected 0.314 as BGS Bright
Check: 0.039
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3mtl-bright-hp-2287.ecsv
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3zs-bright-hp-2287.ecsv
Selected 0.293 as BGS Bright
Check: 0.033
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3mtl-bright-hp-2333.ecsv
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3zs-bright-hp-2333.ecsv
Selected 0.331 as BGS Bright
Check: 0.039
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3mtl-bright-hp-2334.ecsv
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3zs-bright-hp-2334.ecsv
Selected 0.341 as BGS Bright
Check: 0.038
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3mtl-bright-hp-2335.ecsv
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv3zs-bright-hp-2335.ecsv
Selected 0.332 as BGS Bright
Check: 0.038
Writing /global/cscratch1/sd/mjwilson/altmtls/ledger/initial/v3/sv

In [37]:
is_mws= ((init['SV3_DESI_TARGET'] & desi_mask['MWS_ANY']) != 0)
init[~is_mws]

RA,DEC,PARALLAX,PMRA,PMDEC,REF_EPOCH,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,PRIORITY,NUMOBS,NUMOBS_MORE,Z,ZWARN,TIMESTAMP,VERSION,TARGET_STATE,ZTILEID,SV3_SCND_TARGET
deg,deg,mas,mas / yr,mas / yr,yr,,,,,,,,,,,,,,s,,,,
float64,float64,float32,float32,float32,float32,int64,int64,int64,int64,float64,int32,int64,int64,int64,int64,int64,float64,int64,str25,str14,str30,int32,int64
192.88653564453125,28.34505271911621,0.0,0.0,0.0,0.0,0,0,0,288230405667028992,0.27306271420512496,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.3077392578125,27.24908447265625,0.0,0.0,0.0,0.0,0,0,0,288230405667028993,0.014055136555883085,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.9054718017578,27.307897567749023,0.0,0.0,0.0,0.0,1152921504606846976,257,0,288230405667028994,0.9241744019501371,516,102000,3,102000,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.91357421875,27.774377822875977,0.0,0.0,0.0,0.0,0,0,0,288230405667028995,0.9852042476006285,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8726043701172,27.83574104309082,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028996,0.8098146725941684,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8536376953125,27.830402374267578,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028997,0.15176595252535818,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.2030487060547,27.730819702148438,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028998,0.2440701113007242,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8957061767578,27.703933715820312,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028999,0.06202969481519138,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0


In [38]:
init

RA,DEC,PARALLAX,PMRA,PMDEC,REF_EPOCH,SV3_DESI_TARGET,SV3_BGS_TARGET,SV3_MWS_TARGET,TARGETID,SUBPRIORITY,OBSCONDITIONS,PRIORITY_INIT,NUMOBS_INIT,PRIORITY,NUMOBS,NUMOBS_MORE,Z,ZWARN,TIMESTAMP,VERSION,TARGET_STATE,ZTILEID,SV3_SCND_TARGET
deg,deg,mas,mas / yr,mas / yr,yr,,,,,,,,,,,,,,s,,,,
float64,float64,float32,float32,float32,float32,int64,int64,int64,int64,float64,int32,int64,int64,int64,int64,int64,float64,int64,str25,str14,str30,int32,int64
192.88653564453125,28.34505271911621,0.0,0.0,0.0,0.0,0,0,0,288230405667028992,0.27306271420512496,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.3077392578125,27.24908447265625,0.0,0.0,0.0,0.0,0,0,0,288230405667028993,0.014055136555883085,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.9054718017578,27.307897567749023,0.0,0.0,0.0,0.0,1152921504606846976,257,0,288230405667028994,0.9241744019501371,516,102000,3,102000,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.91357421875,27.774377822875977,0.0,0.0,0.0,0.0,0,0,0,288230405667028995,0.9852042476006285,516,-99,3,-99,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8726043701172,27.83574104309082,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028996,0.8098146725941684,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8536376953125,27.830402374267578,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028997,0.15176595252535818,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
193.2030487060547,27.730819702148438,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028998,0.2440701113007242,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
192.8957061767578,27.703933715820312,0.0,0.0,0.0,0.0,1152921504606846976,514,0,288230405667028999,0.06202969481519138,516,102100,3,102100,0,3,-1.0,-1,2021-04-04T23:05:09,0.57.0,BGS|UNOBS,-1,0
